# RESUMEN: 

### En forma general el modelo se basa en dos partes, la primera en encontrar los libros similares basándonos en las reseñas y la segunda partes es recomendar los libros similares con mejores calificaciones. 

# 1) EXPLORACIÓN DE LOS DATOS 

In [2]:
import pandas as pd

En mi computaor la carga de los datos completos se demora 20min, por esto voy a disminuir el tamaño de los dataframes para poder ilustrar el proceso

In [49]:
books = pd.read_csv('books_data.csv', nrows=6000) #cargamos solamente las primeras 100000 filas de los dataframes
ratings = pd.read_csv('Books_rating.csv', nrows=6000)

In [50]:
books.head()


,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [52]:
ratings.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


### Selección de variables de interés

De los dos dataframes anteriores identificamos que las variables más importantes para realizar el modelo de recomendación son: el número de número de identificación del libro, el título del libro, la calificación del libro y la reseña del libro. Vamos a descartar el dataframe books, pues si bien proporciona una variable de interés que es "description" esta variable se puede encontrar también en "review/text" del dataframe ratings. 

## 1.1) Exploremos la cantidad de valores faltantes

In [53]:
ratings=ratings[['Id', 'Title', 'review/score', 'review/text']]
print(ratings.isnull().sum())

Id              0
Title           0
review/score    0
review/text     0
dtype: int64


# 2) VECTORIZAR LAS RESEÑAS
###  Ahora convirtamos las reseñas en vectores numéricos y calculemos la similitud usando el **criterio de coseno, la distancia euclidiana y la distancia de manhattan**, después de esto vamos a determinar cuáles son los libros más similares a un libro en particular. Luego la remendación la realizamos basandos en los libros más similares.


In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances

tfidf = TfidfVectorizer(stop_words='english') # stop english significa que se eliminaran las palabras que no aportan informacion al análisis
tfidf_matrix = tfidf.fit_transform(ratings['review/text']) # se crea la matriz de tfidf con los textos convertidos a números

cosine_sim = cosine_similarity(tfidf_matrix) # se calcula la similitud coseno de la matriz de tfidf para obtener los libros más similares entre sí
#de acuerdo a su reseña. 
euclidean_dist = euclidean_distances(tfidf_matrix) # se calcula la distancia euclidiana de la matriz de tfidf para obtener los libros más similares entre sí
manhattan_dist = manhattan_distances(tfidf_matrix) # se calcula la distancia manhattan de la matriz de tfidf para obtener los libros más similares entre sí


# 3) MODELOS DE RECOMENDACIÓN BASADO SÓLO EN LAS RESEÑAS

## En esta parte del modelo sólo nos basamos en los reviews para encontrar los libros más similares, sin importar la calificación, eso lo harémos más adelante. 

In [95]:
def get_recommendations(book_title, u, n=3):

    # Asegurarse de que el título del libro existe en el DataFrame, esto pues en mi caso no tomé todo el dataframe. 

    if book_title not in ratings['Title'].values:
        return []

    # Encuentra el ID del libro objetivo basado en el título
    book_id = ratings[ratings['Title'] == book_title].iloc[0]['Id']

    # Encuentra el índice del libro objetivo en el DataFrame
    book_index = ratings[ratings['Id'] == book_id].index[0]

    # Calcula las similitudes de las reseñas con diferentes métodos
    if u==1: 
        sim_scores = list(enumerate(cosine_sim[book_index]))
    if u==2: 
        sim_scores = list(enumerate(euclidean_dist[book_index]))
    if u==3: 
        sim_scores = list(enumerate(manhattan_dist[book_index]))
    

    # Ordena los libros por similitud en orden descendente
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Excluye el propio libro de la lista y selecciona los n libros más similares
    sim_scores = sim_scores[1:n+1]

    # Obtiene los IDs de los libros más similares
    similar_books = [ratings['Id'][i[0]] for i in sim_scores]

    # Obtiene los nombres de los libros recomendados   

    rec_books = []
    for book in similar_books:
        book_name = ratings[ratings['Id'] == book].iloc[0]['Title']
        rec_books.append(book_name)

    return rec_books


In [117]:
libros=['Its Only Art If Its Well Hung!', 'Wonderful Worship in Smaller Churches' ]


for u in [1, 2, 3]:
       print('\n')
       print('*********** método {}******************'.format(u))
       
       for l in libros:
              print('recomendaciones de acuerdo al libro:', l)
              print(get_recommendations(l, u, n=3))
              print('\n')
       print('***************************************')



*********** método 1******************
recomendaciones de acuerdo al libro: Its Only Art If Its Well Hung!
['The Scarlet Letter A Romance', 'Mothman Prophecies', 'Cruel and Unusual (G K Hall Large Print Book Series (Cloth))']


recomendaciones de acuerdo al libro: Wonderful Worship in Smaller Churches
['Wonderful Worship in Smaller Churches', 'Wonderful Worship in Smaller Churches', 'The Gifted Adult: A Revolutionary Guide for Liberating Everyday Genius(tm)']


***************************************


*********** método 2******************
recomendaciones de acuerdo al libro: Its Only Art If Its Well Hung!
['Eyewitness Travel Guide to Europe', "Dealing With Disappointment: Helping Kids Cope When Things Don't Go Their Way", 'Cien Aos de BOXEO (One Hundred Years of Box)']


recomendaciones de acuerdo al libro: Wonderful Worship in Smaller Churches
['Vector Quantization and Signal Compression (The Springer International Series in Engineering and Computer Science)', 'Voices from the Far

# 4) MODELOS DE RECOMENDACIÓN BASADO EN CALIFICACIÓN NUMÉRICA.

## En esta parte del modelo nos centramos solamente en las calificaciones de los libros, no es sus reseñas. 

El paquete surprise es una biblioteca de Python especializada en la construcción y evaluación de sistemas de recomendación. 

Usaremos los 3 modelos de recomendación más usados de la librería surprise: KNNBasic, SVD y  SlopeOne. Luego comparemos su desempeño usando diferentes métricas como mae,  rmse y mse:

RMSE (Root Mean Squared Error): Mide la raíz cuadrada del error cuadrático medio entre las predicciones y los valores reales. Una métrica útil para entender la magnitud del error en las mismas unidades que las calificaciones originales.

MAE (Mean Absolute Error): Mide el error absoluto promedio entre las predicciones y los valores reales. Proporciona una medida de la precisión del modelo en términos de desviación media de las predicciones.

MSE (Mean Squared Error): Mide el promedio de los errores cuadrados entre las predicciones y los valores reales. Similar al RMSE pero sin la raíz cuadrada, es más sensible a los grandes errores debido al efecto cuadrático.


In [9]:
#prepraramos los datos para el modelo de recomendación 
from surprise import Dataset, Reader, KNNBasic, SlopeOne, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['Id','Title' ,'review/score']], reader)

trainset, testset = train_test_split(data, test_size=0.25)

## 4.1) KNNBasic

In [10]:
sim_options = {
    'name': 'cosine',
    'user_based': False  # Calcular similitudes entre ítems
}


from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score



modelo1 = KNNBasic(sim_options=sim_options)
modelo1.fit(trainset)

# Evaluar el algoritmo
predictions = modelo1.test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)
accuracy.mse(predictions)



Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.2112
MAE:  0.9038
MSE: 1.4669


1.4668965272292076

## 4.2) SVD

In [11]:
# Crear el modelo SVD
modelo2 = SVD()

# Entrenar el modelo con los datos de entrenamiento
modelo2.fit(trainset)

# Evaluar el modelo con los datos de prueba
predictions2 = modelo2.test(testset)

# Calcular y mostrar las métricas de evaluación
rmse = accuracy.rmse(predictions2)
mae = accuracy.mae(predictions2)
mse = accuracy.mse(predictions2)

RMSE: 1.1916
MAE:  0.9154
MSE: 1.4200


## 4.3) SlopeOne

In [12]:
# Crear el modelo Slope One
modelo3 = SlopeOne()

# Entrenar el modelo con los datos de entrenamiento
modelo3.fit(trainset)

# Evaluar el modelo con los datos de prueba
predictions3 = modelo3.test(testset)

# Calcular y mostrar las métricas de evaluación
rmse = accuracy.rmse(predictions3)
mae = accuracy.mae(predictions3)
mse = accuracy.mse(predictions3)

RMSE: 1.2083
MAE:  0.9137
MSE: 1.4600


# 5)  APLICACIÓN DEL MODELO COLABORATIVO, ES DECIR, BASADO EN REVIEWS Y CALIFICACIONES

## Seleccionamos el modelo que nos dio mejores métricas (SVD) y lo combinamos con el modelo de recomendación basado en los reviews, para seleccionar los libros similares y que además puedan tener las calificaciones más altas. 

In [102]:
# Recomendaciones basadas en el modelo de filtrado colaborativo
def get_collab_recommendations(book_title, n=3):

    return 0

# 